In [1]:
import pandas as pd
import numpy as np
import math
import mpmath

In [68]:
#rewards = pd.read_csv('/Users/dulpneto/workspace/OpenAI-GYM-CarRacing-DQN-Risk/r_bound_300/rewards_2.csv')
rewards = pd.read_csv('/Users/dulpneto/workspace/OpenAI-GYM-CarRacing-DQN-Risk/rewards_fixed_policy.log')

lambs = []
for i in range(-10,11):
    lambs.append(i/10)
    
#for i in range(0,11):
#    lambs.append(round(0.7726278200644+(i/100000000000000), 14))


for lamb in lambs:
    if lamb == 0:
        rewards['{}'.format(lamb)] = rewards.apply(lambda row : row[1]/1000, axis = 1)
        continue
    try:
        rewards['{}'.format(lamb)] = rewards.apply(lambda row : np.sign(lamb) * np.exp(lamb * row[1]), axis = 1)
    except (ValueError, OverflowError) as error:
        print(error)
        print('Error {}'.format(lamb))

rewards

,policy,reward,-1.0,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,...,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
0,1,-80.5,-9.134942e+34,-2.914978e+31,-9.301749e+27,-2.968206e+24,-9.471603e+20,-3.022407e+17,-9.644558e+13,-3.077597e+10,...,0.000319,1.018260e-07,3.249288e-11,1.036854e-14,3.308622e-18,1.055788e-21,3.369038e-25,1.075067e-28,3.430558e-32,1.094698e-35
1,1,-15.7,-6.582993e+06,-1.369560e+06,-2.849303e+05,-5.927838e+04,-1.233258e+04,-2.565734e+03,-5.337887e+02,-1.110522e+02,...,0.208045,4.328280e-02,9.004778e-03,1.873401e-03,3.897520e-04,8.108602e-05,1.686956e-05,3.509630e-06,7.301616e-07,1.519066e-07
2,1,-80.0,-5.540622e+34,-1.858672e+31,-6.235149e+27,-2.091659e+24,-7.016736e+20,-2.353853e+17,-7.896296e+13,-2.648912e+10,...,0.000335,1.125352e-07,3.775135e-11,1.266417e-14,4.248354e-18,1.425164e-21,4.780893e-25,1.603811e-28,5.380186e-32,1.804851e-35
3,1,-18.7,-1.322229e+08,-2.037868e+07,-3.140837e+06,-4.840774e+05,-7.460777e+04,-1.149882e+04,-1.772241e+03,-2.731442e+02,...,0.154124,2.375410e-02,3.661069e-03,5.642574e-04,8.696542e-05,1.340343e-05,2.065786e-06,3.183864e-07,4.907088e-08,7.562984e-09
4,1,-15.1,-3.612823e+06,-7.981086e+05,-1.763102e+05,-3.894867e+04,-8.604151e+03,-1.900743e+03,-4.198930e+02,-9.275856e+01,...,0.220910,4.880122e-02,1.078068e-02,2.381559e-03,5.261101e-04,1.162230e-04,2.567482e-05,5.671823e-06,1.252962e-06,2.767919e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,5,-25.9,-1.771035e+11,-1.328631e+10,-9.967395e+08,-7.477544e+07,-5.609656e+06,-4.208366e+05,-3.157118e+04,-2.368471e+03,...,0.075020,5.628006e-03,4.222133e-04,3.167446e-05,2.376219e-06,1.782640e-07,1.337338e-08,1.003271e-09,7.526544e-11,5.646417e-12
496,5,-25.9,-1.771035e+11,-1.328631e+10,-9.967395e+08,-7.477544e+07,-5.609656e+06,-4.208366e+05,-3.157118e+04,-2.368471e+03,...,0.075020,5.628006e-03,4.222133e-04,3.167446e-05,2.376219e-06,1.782640e-07,1.337338e-08,1.003271e-09,7.526544e-11,5.646417e-12
497,5,-25.4,-1.074187e+11,-8.471725e+09,-6.681345e+08,-5.269336e+07,-4.155736e+06,-3.277479e+05,-2.584830e+04,-2.038562e+03,...,0.078866,6.219909e-03,4.905418e-04,3.868727e-05,3.051126e-06,2.406313e-07,1.897772e-08,1.496705e-09,1.180397e-10,9.309367e-12
498,5,-25.2,-8.794698e+10,-7.076180e+09,-5.693466e+08,-4.580941e+07,-3.685807e+06,-2.965586e+05,-2.386099e+04,-1.919846e+03,...,0.080460,6.473748e-03,5.208752e-04,4.190942e-05,3.372015e-06,2.713110e-07,2.182958e-08,1.756399e-09,1.413192e-10,1.137049e-11


In [69]:
rewards_by_policy = rewards.groupby('policy')
rewards_by_policy.agg({'reward': ['mean', 'var', 'min', 'max']})

reward                          
          mean          var    min   max
policy                                  
1      -47.207  1860.186920 -212.8 -15.0
2      -18.454     0.472408  -20.4 -17.3
3      -67.026  3455.862954 -307.9 -24.3
4      -25.132     7.642804  -52.2 -24.3
5      -25.885     0.200480  -27.1 -25.2

In [70]:
agg = {}
for lamb in lambs:
    agg['{}'.format(lamb)] = ['mean']
        
def highlight_max(s):
    is_max = s == s.max()
    return ['background: lightgreen' if cell else '' for cell in is_max]        
rewards_by_policy_formated = rewards_by_policy.agg(agg).style.format('{:.2e}').highlight_max(color='lightgreen')
rewards_by_policy_formated
  

,-1.0,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
policy,,,,,,,,,,,,,,,,,,,,,
1,-2.62e+90,-1.50e+81,-8.60e+71,-4.93e+62,-2.83e+53,-1.63e+44,-9.40e+34,-5.53e+25,-3.45e+16,-2.58e+07,-4.72e-02,9.31e-02,1.80e-02,3.73e-03,7.85e-04,1.66e-04,3.53e-05,7.52e-06,1.60e-06,3.43e-07,7.35e-08
2,-1.33e+08,-2.00e+07,-3.03e+06,-4.60e+05,-7.04e+04,-1.08e+04,-1.67e+03,-2.59e+02,-4.05e+01,-6.35e+00,-1.85e-02,1.58e-01,2.52e-02,4.02e-03,6.45e-04,1.04e-04,1.68e-05,2.72e-06,4.44e-07,7.24e-08,1.19e-08
3,-5.24e+131,-2.23e+118,-9.45e+104,-4.01e+91,-1.70e+78,-7.24e+64,-3.07e+51,-1.31e+38,-5.64e+24,-2.68e+11,-6.70e-02,3.55e-02,2.64e-03,2.05e-04,1.61e-05,1.28e-06,1.03e-07,8.34e-09,6.80e-10,5.58e-11,4.59e-12
4,-4.68e+20,-2.53e+18,-1.37e+16,-7.40e+13,-4.00e+11,-2.16e+09,-1.17e+07,-6.50e+04,-4.85e+02,-1.38e+01,-2.51e-02,8.25e-02,6.88e-03,5.75e-04,4.82e-05,4.04e-06,3.39e-07,2.85e-08,2.40e-09,2.02e-10,1.71e-11
5,-1.94e+11,-1.43e+10,-1.05e+09,-7.79e+07,-5.77e+06,-4.29e+05,-3.19e+04,-2.38e+03,-1.78e+02,-1.33e+01,-2.59e-02,7.52e-02,5.67e-03,4.28e-04,3.24e-05,2.45e-06,1.86e-07,1.41e-08,1.08e-09,8.20e-11,6.26e-12
